In [1]:
# Import packages and custom functions
%run 'create_final_csv_functions.py'

In [ ]:
# CREATE CSV WITH NOISE AND METEO DATA 
months = ["Jan", "Feb", "March", "April","May", "June", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]   
for m in months: 
    create_final_csv_by_month(m)

# Concatenate all months in one csv
concatenate_noise_sub_csv()
# The csv was uploaded to s3 bucket

In [35]:
# ADD TO PREVIOUS CSV ONE COLUMN WITH COUNT BARS OPEN USING DATA FROM GOOGLE MAPS API 
# Read datasets to merge
df = pd.read_csv("s3://mda.project.monaco/project_data_v1.csv") 
df_openings = pd.read_csv("s3://mda.project.monaco/openings.csv")

# Delete dulci_open variable 
df = df.drop('dulci_open', axis=1)

# Rename day into day_of_week
df_openings = df_openings.rename(columns={"day": "day_of_week"})

# Add to df a column "location_name" that can be used as key when joining
for i, row in df.iterrows(): 
    if "01" in row["description"]:
        df.at[i,'location_name'] = "01"
    elif "02" in row["description"]:
        df.at[i,'location_name'] = "02"
    elif "03" in row["description"]:
        df.at[i,'location_name'] = "03"
    elif "04" in row["description"]:
        df.at[i,'location_name'] = "04"
    elif "05" in row["description"]:
        df.at[i,'location_name'] = "05"
    elif "06" in row["description"]:
        df.at[i,'location_name'] = "06"
    elif "07" in row["description"]:
        df.at[i,'location_name'] = "07"
    elif "Vrijthof" in row["description"]:
        df.at[i,'location_name'] = "Vrijthof"

# Merge 
df_merged = pd.merge(left = df, right=df_openings, on = ["day_of_week", "hour", "location_name"])

# Remove location_name variable
df_merged = df_merged.drop('location_name', axis=1)

# Save dataset and upload to s3 bucket at: "s3://mda.project.monaco/project_data.csv"
df_merged.to_csv("data/project_data.csv", index=False) 